In [ ]:
%load_ext dockermagic

# HDFS

## HDFS - Web Interface

- Master node
    - NameNode: http://localhost:9870
    - Secondary NameNode: http://localhost:9868
- Worker node
    - hadoop1
        - DataNode: http://localhost:9864
    - hadoop2
        - DataNode: http://localhost:9865
    - hadoop3
        - DataNode: http://localhost:9866

## HDFS - CLI

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh
hdfs help

## Administration Commands

- https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-hdfs/HDFSCommands.html

### Verify HDFS cluster status

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

# print topology
hdfs dfsadmin -printTopology

printf "\n%40s\n\n" |tr " " "="

hdfs dfsadmin -report

### Replication factor

#### Run randomtext application

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

# deletes randomtext folder if it exists
hdfs dfs -test -d /user/hadoop/randomtext && hdfs dfs -rm -r /user/hadoop/randomtext

cd /opt/hadoop/share/hadoop/mapreduce

hadoop jar ./hadoop-mapreduce-examples-$HADOOP_VERSION.jar randomtextwriter \
  -D mapreduce.randomtextwriter.totalbytes=157286400 \
  -D mapreduce.randomtextwriter.bytespermap=52428800 \
  -D mapreduce.output.fileoutputformat.compress=false \
  -outFormat org.apache.hadoop.mapreduce.lib.output.TextOutputFormat \
  randomtext

#### List folder block location

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs fsck /user/hadoop/randomtext -files -blocks -locations

#### Change replication factor of all files in directory to 3

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -setrep 3 /user/hadoop/randomtext

#### List folder block location

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs fsck /user/hadoop/randomtext -files -blocks -locations

#### Change replication factor back to 2

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -setrep 2 /user/hadoop/randomtext

#### List folder block location

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs fsck /user/hadoop/randomtext -files -blocks -locations

### Decomission nodes

- dfs.hosts.exclude in hdfs-site.xml

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

# Decomissioning hadoop1
cat > /opt/hadoop/etc/hadoop/dfs.exclude << EOF
hadoop1
EOF

hdfs dfsadmin -refreshNodes

- **Namenode:** http://localhost:9870

#### Report HDFS Status

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfsadmin -report

#### Recomission all nodes

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

cat > /opt/hadoop/etc/hadoop/dfs.exclude << EOF
EOF

hdfs dfsadmin -refreshNodes

#### Report HDFS status

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfsadmin -report

### Handling datanode failures

- timeouts defined in hdfs-site.xml 
    - dfs.namenode.heartbeat.recheck-interval = 10000 (10 seconds)
    - dfs.heartbeat.interval = 3 seconds
- timeout = 2 x recheck-interval + 10 x heartbeat.interval
    - timeout = 50 seconds

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

# get dfs.namenode.heartbeat.recheck-interval
hdfs getconf -confKey dfs.namenode.heartbeat.recheck-interval

# get dfs.heartbeat.interval
hdfs getconf -confKey dfs.heartbeat.interval

#### Simulate node fault

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

ssh hadoop1 'kill -9 $(cat /tmp/hadoop-hadoop-datanode.pid)'

- **Namenode:** http://localhost:9870

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfsadmin -report

#### Restart nodemanager

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

ssh hadoop1 /opt/hadoop/bin/hdfs --daemon start datanode

#### Refresh nodes

In [ ]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfsadmin -refreshNodes
hdfs dfsadmin -report